# Exoplanet Detection - FITS-Only Features Model

This version trains on:
1. **Light curve time series** (flux values)
2. **Features extracted from FITS file only** (no external KOI table features)

**Key advantage**: Model can predict using only a FITS file - no need for KOI catalog features!

In [1]:
import os, glob, re, time, math, random
from pathlib import Path

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

import lightkurve as lk
from astropy.stats import sigma_clip
from scipy import signal
from scipy.stats import skew, kurtosis

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, 
    confusion_matrix, roc_curve, auc, precision_recall_curve
)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Device selection
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print("Using device:", DEVICE)

Using device: mps


In [2]:
# Configuration
LIGHTCURVE_DIR = "../lightcurves/"
KOI_CSV = "../dataset/koi.csv"  # Only used for labels, not features
CACHE_DIR = "lc_cache_fits_only/"
FEATURES_CACHE = "features_cache_fits_only/"
SEQ_LEN = 2000
BATCH_SIZE = 16
EPOCHS = 25
RANDOM_SEED = 42
MODEL_OUT = "exoplanet_fits_only.pth"
SCALER_OUT = "fits_scaler.joblib"
FEATURE_NAMES_OUT = "fits_feature_names.joblib"

os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(FEATURES_CACHE, exist_ok=True)

# Set seeds
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if DEVICE.type == "cuda":
    torch.cuda.manual_seed_all(RANDOM_SEED)

## Feature Extraction from FITS Files

We'll extract features that can be computed directly from the light curve:

### Statistical Features:
- Mean, median, std, min, max flux
- Skewness, kurtosis
- Percentiles (25th, 75th)

### Time-domain Features:
- Number of data points
- Time span
- Cadence (median time difference)

### Variability Features:
- Variance
- Range (max - min)
- Coefficient of variation
- MAD (Median Absolute Deviation)

### Frequency-domain Features:
- Dominant frequency (from periodogram)
- Power at dominant frequency

### Transit-related Features:
- Number of significant dips (sigma-clipped outliers)
- Depth of deepest dip
- Mean dip depth

In [3]:
def extract_fits_features(fits_path):
    """
    Extract features directly from a FITS file that can be computed
    without external catalog information.
    
    Returns:
        dict: Feature dictionary
    """
    features = {}
    
    try:
        lc = lk.read(fits_path)
        
        # Get flux array
        if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
            flux = lc.PDCSAP_FLUX.value
        elif hasattr(lc, "SAP_FLUX") and lc.SAP_FLUX is not None:
            flux = lc.SAP_FLUX.value
        else:
            flux = lc.flux.value if hasattr(lc, 'flux') else np.array([])
        
        # Get time array
        time = lc.time.value if hasattr(lc, 'time') else np.arange(len(flux))
        
        # Remove NaNs
        valid = ~np.isnan(flux) & ~np.isnan(time)
        flux = flux[valid]
        time = time[valid]
        
        if len(flux) == 0:
            return get_default_features()
        
        # Normalize flux by median
        median_flux = np.nanmedian(flux)
        if median_flux > 0:
            flux_norm = flux / median_flux
        else:
            flux_norm = flux
        
        # === STATISTICAL FEATURES ===
        features['flux_mean'] = np.mean(flux_norm)
        features['flux_median'] = np.median(flux_norm)
        features['flux_std'] = np.std(flux_norm)
        features['flux_min'] = np.min(flux_norm)
        features['flux_max'] = np.max(flux_norm)
        features['flux_range'] = features['flux_max'] - features['flux_min']
        features['flux_skew'] = skew(flux_norm)
        features['flux_kurtosis'] = kurtosis(flux_norm)
        features['flux_p25'] = np.percentile(flux_norm, 25)
        features['flux_p75'] = np.percentile(flux_norm, 75)
        
        # === VARIABILITY FEATURES ===
        features['flux_var'] = np.var(flux_norm)
        features['flux_cv'] = features['flux_std'] / features['flux_mean'] if features['flux_mean'] != 0 else 0
        features['flux_mad'] = np.median(np.abs(flux_norm - features['flux_median']))
        
        # === TIME FEATURES ===
        features['n_points'] = len(flux)
        features['time_span'] = np.max(time) - np.min(time) if len(time) > 1 else 0
        if len(time) > 1:
            time_diff = np.diff(time)
            features['cadence_median'] = np.median(time_diff)
            features['cadence_std'] = np.std(time_diff)
        else:
            features['cadence_median'] = 0
            features['cadence_std'] = 0
        
        # === TRANSIT/DIP FEATURES ===
        # Identify significant dips using sigma clipping
        try:
            clipped = sigma_clip(flux_norm, sigma=3, maxiters=5)
            dips = flux_norm < (features['flux_median'] - 2 * features['flux_std'])
            features['n_dips'] = np.sum(dips)
            
            if features['n_dips'] > 0:
                dip_depths = features['flux_median'] - flux_norm[dips]
                features['max_dip_depth'] = np.max(dip_depths)
                features['mean_dip_depth'] = np.mean(dip_depths)
            else:
                features['max_dip_depth'] = 0
                features['mean_dip_depth'] = 0
        except:
            features['n_dips'] = 0
            features['max_dip_depth'] = 0
            features['mean_dip_depth'] = 0
        
        # === FREQUENCY FEATURES ===
        try:
            if len(flux_norm) > 10:
                # Compute periodogram
                freqs, power = signal.periodogram(flux_norm, fs=1.0/features['cadence_median'] if features['cadence_median'] > 0 else 1.0)
                
                # Find dominant frequency (excluding DC component)
                if len(power) > 1:
                    dom_idx = np.argmax(power[1:]) + 1
                    features['dominant_freq'] = freqs[dom_idx]
                    features['dominant_power'] = power[dom_idx]
                    features['power_ratio'] = power[dom_idx] / np.mean(power[1:]) if np.mean(power[1:]) > 0 else 0
                else:
                    features['dominant_freq'] = 0
                    features['dominant_power'] = 0
                    features['power_ratio'] = 0
            else:
                features['dominant_freq'] = 0
                features['dominant_power'] = 0
                features['power_ratio'] = 0
        except:
            features['dominant_freq'] = 0
            features['dominant_power'] = 0
            features['power_ratio'] = 0
        
        # === AUTOCORRELATION FEATURE ===
        try:
            if len(flux_norm) > 50:
                # Autocorrelation at lag 1
                acf_1 = np.corrcoef(flux_norm[:-1], flux_norm[1:])[0, 1]
                features['autocorr_1'] = acf_1 if np.isfinite(acf_1) else 0
            else:
                features['autocorr_1'] = 0
        except:
            features['autocorr_1'] = 0
            
    except Exception as e:
        print(f"Error extracting features from {fits_path}: {e}")
        return get_default_features()
    
    return features


def get_default_features():
    """Return default feature dict for failed extractions"""
    return {
        'flux_mean': 1.0, 'flux_median': 1.0, 'flux_std': 0.0,
        'flux_min': 1.0, 'flux_max': 1.0, 'flux_range': 0.0,
        'flux_skew': 0.0, 'flux_kurtosis': 0.0,
        'flux_p25': 1.0, 'flux_p75': 1.0,
        'flux_var': 0.0, 'flux_cv': 0.0, 'flux_mad': 0.0,
        'n_points': 0, 'time_span': 0.0,
        'cadence_median': 0.0, 'cadence_std': 0.0,
        'n_dips': 0, 'max_dip_depth': 0.0, 'mean_dip_depth': 0.0,
        'dominant_freq': 0.0, 'dominant_power': 0.0, 'power_ratio': 0.0,
        'autocorr_1': 0.0
    }

print("Feature extraction functions defined.")
print("Total features:", len(get_default_features()))

Feature extraction functions defined.
Total features: 24


In [4]:
# Load KOI data (for labels and kepid only)
koi = pd.read_csv(KOI_CSV)
print(f"KOI rows: {len(koi)}")

# Create label mapping
disp_map = {"FALSE POSITIVE": 0, "CANDIDATE": 1, "CONFIRMED": 2}
koi["label"] = koi["koi_disposition"].map(disp_map).astype("Int64")

print("Label distribution:")
print(koi["label"].value_counts().sort_index())

# Map FITS files
all_fits = glob.glob(os.path.join(LIGHTCURVE_DIR, "**", "*.fits*"), recursive=True)
print(f"\nTotal FITS files found: {len(all_fits)}")

fits_map = {}
for f in all_fits:
    name = os.path.basename(f)
    m = re.search(r'(\d{6,9})', name)
    if m:
        try:
            fits_map[int(m.group(1))] = f
        except:
            pass

def match_fits_for_kepid(kepid):
    try:
        k = int(float(kepid))
    except:
        return None
    if k in fits_map:
        return fits_map[k]
    for pad in (7,8,9):
        kp = int(str(k).zfill(pad))
        if kp in fits_map:
            return fits_map[kp]
    return None

koi['fits_path'] = koi['kepid'].apply(lambda x: match_fits_for_kepid(x) if not pd.isna(x) else None)
koi = koi[~koi['fits_path'].isna()].reset_index(drop=True)
print(f"KOI rows with matched FITS: {len(koi)}")

KOI rows: 9564
Label distribution:
label
0    4839
1    1979
2    2746
Name: count, dtype: Int64

Total FITS files found: 8214
KOI rows with matched FITS: 9564


In [5]:
# Extract or load cached features
print("Extracting FITS-based features...")

features_list = []
for idx, row in koi.iterrows():
    kepid = row['kepid']
    fits_path = row['fits_path']
    
    # Check cache
    cache_file = os.path.join(FEATURES_CACHE, f"{kepid}.npz")
    
    if os.path.exists(cache_file):
        loaded = np.load(cache_file)
        feats = {k: loaded[k].item() for k in loaded.files}
    else:
        feats = extract_fits_features(fits_path)
        # Save to cache
        np.savez(cache_file, **feats)
    
    features_list.append(feats)
    
    if (idx + 1) % 100 == 0:
        print(f"Processed {idx + 1}/{len(koi)} files...")

# Convert to DataFrame
features_df = pd.DataFrame(features_list)
print(f"\nExtracted {len(features_df.columns)} features")
print("Feature columns:", list(features_df.columns))

# Handle any remaining NaNs
features_df = features_df.fillna(0)

print("\nFeature statistics:")
print(features_df.describe())

Extracting FITS-based features...
Error extracting features from ../lightcurves/10797460.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10811496.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10848459.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10854555.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10872983.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/6721123.fits: 'pdcsap_flux'


/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.P

Error extracting features from ../lightcurves/10910878.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/11446443.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10666592.fits: 'pdcsap_flux'


/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.P

Error extracting features from ../lightcurves/6922244.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10984090.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10419211.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10464078.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10480982.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10485250.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10526549.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10583066.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10583180.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10601284.fits: 'pdcsap_flux'


...

Error extracting features from ../lightcurves/12117215.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/12168280.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/9912977.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/9938836.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/9941425.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/9959368.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/9959765.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10011213.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10014875.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/3954211.fits: 'pdcsap_flux'


/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.P

Error extracting features from ../lightcurves/4056579.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/4276445.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/4371172.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/4557341.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/4645492.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/4738095.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10023069.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10028127.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10079317.fits: 'pdcsap_flux'


/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1710533046.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.P

Error extracting features from ../lightcurves/10079481.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10081899.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10199817.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10205598.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10031643.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10090151.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10128825.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10147276.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10155286.fits: 'pdcsap_flux'
Error extracting features from ../lightcurves/10156110.fits: 'pdcsap_flux'

Extracted 24 features
Feature columns: ['flux_mean', 'flux_median', 'flux_std', 'flux_min', 'flux_max', 'flux_range', 'flux_skew', 'flux_kurtosis', 'flux_p25', 'flux_p75', 'flux_var', 'flux_cv', 'flux_mad', 'n_points', 'time_span', 'cadence_median',

In [6]:
# Prepare data
X_tab = features_df.values.astype(np.float32)
y_arr = koi['label'].values.astype(np.float32)
fits_paths = koi['fits_path'].values
kepids = koi['kepid'].values

print(f"X_tab shape: {X_tab.shape}")
print(f"y shape: {y_arr.shape}")

# Scale features
scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)

# Save scaler and feature names
joblib.dump(scaler, SCALER_OUT)
joblib.dump(list(features_df.columns), FEATURE_NAMES_OUT)
print(f"\nSaved scaler to {SCALER_OUT}")
print(f"Saved feature names to {FEATURE_NAMES_OUT}")

X_tab shape: (9564, 24)
y shape: (9564,)

Saved scaler to fits_scaler.joblib
Saved feature names to fits_feature_names.joblib


In [7]:
# Light curve processing function (same as before)
def process_and_cache_flux(fits_path, seq_len=SEQ_LEN, cache_dir=CACHE_DIR):
    base_match = re.search(r'(\d{6,9})', os.path.basename(fits_path))
    if base_match:
        base = base_match.group(1)
    else:
        base = Path(fits_path).stem
    cache_path = os.path.join(cache_dir, f"{base}.npy")
    
    if os.path.exists(cache_path):
        return np.load(cache_path)

    try:
        lc = lk.read(fits_path)
        if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
            arr = lc.PDCSAP_FLUX.value
        elif hasattr(lc, "SAP_FLUX") and lc.SAP_FLUX is not None:
            arr = lc.SAP_FLUX.value
        else:
            arr = lc.flux.value if hasattr(lc, 'flux') else np.zeros(seq_len)
    except:
        arr = np.ones(seq_len, dtype=np.float32)
        np.save(cache_path, arr)
        return arr

    arr = np.asarray(arr, dtype=np.float32)
    if np.all(np.isnan(arr)) or np.nanmax(np.abs(arr)) == 0:
        arr = np.ones(seq_len, dtype=np.float32)
    else:
        med = np.nanmedian(arr)
        if not np.isfinite(med) or med == 0:
            med = 1.0
        arr = np.nan_to_num(arr / med, nan=0.0, posinf=0.0, neginf=0.0)

    if len(arr) < seq_len:
        padded = np.ones(seq_len, dtype=np.float32)
        padded[:len(arr)] = arr
        arr = padded
    else:
        arr = arr[:seq_len]

    np.save(cache_path, arr)
    return arr

In [8]:
# Dataset class
class ExoplanetDatasetFITSOnly(Dataset):
    def __init__(self, fits_paths, X_tab_scaled, y_arr, indices, seq_len=SEQ_LEN, cache_dir=CACHE_DIR):
        self.fits = np.array(fits_paths)[indices]
        self.tab = X_tab_scaled[indices]
        self.y = y_arr[indices]
        self.seq_len = seq_len
        self.cache_dir = cache_dir

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        flux = process_and_cache_flux(self.fits[idx], seq_len=self.seq_len, cache_dir=self.cache_dir)
        lc_tensor = torch.from_numpy(flux).float().unsqueeze(0)
        tab_tensor = torch.from_numpy(self.tab[idx]).float()
        label = torch.tensor(self.y[idx], dtype=torch.float32)
        return lc_tensor, tab_tensor, label

# Train/val split by kepid
unique_kepids = np.unique(kepids)
train_kepids, val_kepids = train_test_split(unique_kepids, test_size=0.2, random_state=RANDOM_SEED)
train_mask = np.isin(kepids, train_kepids)
val_mask = np.isin(kepids, val_kepids)
train_idx = np.where(train_mask)[0]
val_idx = np.where(val_mask)[0]

print(f"Train samples: {len(train_idx)}")
print(f"Validation samples: {len(val_idx)}")

train_ds = ExoplanetDatasetFITSOnly(fits_paths, X_tab_scaled, y_arr, train_idx, seq_len=SEQ_LEN, cache_dir=CACHE_DIR)
val_ds = ExoplanetDatasetFITSOnly(fits_paths, X_tab_scaled, y_arr, val_idx, seq_len=SEQ_LEN, cache_dir=CACHE_DIR)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Check batch shapes
lc_b, tab_b, y_b = next(iter(train_loader))
print(f"\nBatch shapes: LC={lc_b.shape}, Tab={tab_b.shape}, Y={y_b.shape}")

Train samples: 7662
Validation samples: 1902


/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.P


Batch shapes: LC=torch.Size([16, 1, 2000]), Tab=torch.Size([16, 24]), Y=torch.Size([16])


/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.PDCSAP_FLUX is not None:
/var/folders/9z/sbfz5w7j381_x61vx7jrhy7m0000gn/T/ipykernel_63637/1747568231.py:15: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  if hasattr(lc, "PDCSAP_FLUX") and lc.P

In [9]:
# Model architecture (same hybrid CNN + MLP)
class HybridExoNetFITSOnly(nn.Module):
    def __init__(self, seq_len, tab_dim, num_classes=3):
        super().__init__()
        # CNN for light curve
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=11, padding=5),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(32, 64, kernel_size=9, padding=4),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Conv1d(64, 128, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten()
        )
        
        self.lc_fc = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # MLP for FITS-derived tabular features
        self.tab_fc = nn.Sequential(
            nn.Linear(tab_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        # Fusion head
        self.head = nn.Sequential(
            nn.Linear(128 + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x_lc, x_tab):
        a = self.cnn(x_lc)
        a = self.lc_fc(a)
        b = self.tab_fc(x_tab)
        c = torch.cat([a, b], dim=1)
        return self.head(c)

tab_dim = X_tab_scaled.shape[1]
model = HybridExoNetFITSOnly(SEQ_LEN, tab_dim, num_classes=3).to(DEVICE)
print(model)
print(f"\nTabular features dimension: {tab_dim}")

HybridExoNetFITSOnly(
  (cnn): Sequential(
    (0): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(9,), stride=(1,), padding=(4,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(64, 128, kernel_size=(7,), stride=(1,), padding=(3,))
    (7): ReLU()
    (8): AdaptiveAvgPool1d(output_size=1)
    (9): Flatten(start_dim=1, end_dim=-1)
  )
  (lc_fc): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (tab_fc): Sequential(
    (0): Linear(in_features=24, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
  )
  (head): Sequential(
    (0): Linear(in_features=160, out_features=64, bias=True)

In [10]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5)

def evaluate_model(model, loader, device=DEVICE):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for lc, tab, y in loader:
            lc, tab = lc.to(device), tab.to(device)
            outputs = model(lc, tab)
            prob = torch.softmax(outputs, dim=1).cpu().numpy()
            ys.append(y.numpy())
            ps.append(prob)
    ys = np.concatenate(ys)
    ps = np.vstack(ps)
    
    acc = accuracy_score(ys, np.argmax(ps, axis=1))
    f1 = f1_score(ys, np.argmax(ps, axis=1), average='weighted')
    report = classification_report(ys, np.argmax(ps, axis=1), 
                                   target_names=["FALSE POSITIVE","CANDIDATE","CONFIRMED"])
    confmat = confusion_matrix(ys, np.argmax(ps, axis=1))
    
    return {"acc": acc, "f1": f1, "report": report, "confmat": confmat, "y": ys, "probs": ps}

print("Training setup complete.")

Training setup complete.


In [12]:
# Training loop
best_val_acc = -1.0
history = {'train_loss': [], 'val_acc': [], 'val_f1': []}

for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0
    t0 = time.time()

    for lc, tab, y in train_loader:
        lc, tab, y = lc.to(DEVICE), tab.to(DEVICE), y.to(DEVICE).long()
        
        optimizer.zero_grad()
        outputs = model(lc, tab)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * lc.size(0)

    epoch_loss /= len(train_loader.dataset)
    history['train_loss'].append(epoch_loss)

    # Validation
    val_stats = evaluate_model(model, val_loader, device=DEVICE)
    val_acc = val_stats["acc"]
    val_f1 = val_stats["f1"]
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    print(f"Epoch {epoch}/{EPOCHS} | Loss: {epoch_loss:.4f} | "
          f"Val_Acc: {val_acc:.4f} | F1: {val_f1:.4f} | "
          f"Time: {time.time()-t0:.1f}s")

    scheduler.step(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), MODEL_OUT)
        print(f"✅ Saved best model (val_acc: {best_val_acc:.4f})")

print(f"\n{'='*60}")
print(f"Training complete! Best validation accuracy: {best_val_acc:.4f}")
print(f"{'='*60}")

KeyboardInterrupt: 

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history['train_loss'], label='Train Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history['val_acc'], label='Val Accuracy', marker='o')
ax2.plot(history['val_f1'], label='Val F1', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Score')
ax2.set_title('Validation Metrics')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Final evaluation
from sklearn.preprocessing import label_binarize

model.load_state_dict(torch.load(MODEL_OUT, map_location=DEVICE))
res = evaluate_model(model, val_loader, device=DEVICE)

print(f"Final Validation Accuracy: {res['acc']:.4f}")
print(f"Final Weighted F1 Score: {res['f1']:.4f}")
print(f"\nClassification Report:\n{res['report']}")
print(f"\nConfusion Matrix:\n{res['confmat']}")

In [ ]:
# ROC Curves
y_true = res['y']
probs = res['probs']

y_true_bin = label_binarize(y_true, classes=[0,1,2])
classes = ["FALSE POSITIVE", "CANDIDATE", "CONFIRMED"]
colors = ['blue','green','red']

plt.figure(figsize=(6,5))
for i, color in enumerate(colors):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2, label=f"{classes[i]} (AUC={roc_auc:.2f})")

plt.plot([0,1], [0,1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - FITS-Only Features Model")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
# Precision-Recall Curves
plt.figure(figsize=(6,5))
for i, color in enumerate(colors):
    precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
    plt.plot(recall, precision, color=color, lw=2, label=classes[i])

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curves - FITS-Only Features")
plt.legend(loc="lower left")
plt.grid(True)
plt.show()

## Create Inference Function

This function can predict using ONLY a FITS file - no KOI table needed!

In [ ]:
def predict_from_fits(fits_path, model, scaler, feature_names, device=DEVICE, seq_len=SEQ_LEN):
    """
    Predict exoplanet classification from a FITS file only.
    
    Args:
        fits_path: Path to FITS file
        model: Trained PyTorch model
        scaler: Fitted StandardScaler
        feature_names: List of feature names
        device: torch device
        seq_len: Sequence length for light curve
    
    Returns:
        dict: Predictions with probabilities
    """
    # Extract features from FITS
    features_dict = extract_fits_features(fits_path)
    
    # Ensure features are in correct order
    features_array = np.array([features_dict[name] for name in feature_names]).reshape(1, -1)
    
    # Scale features
    features_scaled = scaler.transform(features_array)
    
    # Process light curve
    flux = process_and_cache_flux(fits_path, seq_len=seq_len)
    
    # Convert to tensors
    lc_tensor = torch.from_numpy(flux).float().unsqueeze(0).unsqueeze(0).to(device)  # (1, 1, seq_len)
    tab_tensor = torch.from_numpy(features_scaled).float().to(device)  # (1, n_features)
    
    # Predict
    model.eval()
    with torch.no_grad():
        outputs = model(lc_tensor, tab_tensor)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
    
    classes = ["FALSE POSITIVE", "CANDIDATE", "CONFIRMED"]
    prediction = classes[np.argmax(probs)]
    
    return {
        'prediction': prediction,
        'probabilities': dict(zip(classes, probs)),
        'features': features_dict
    }

# Save prediction function
print("Inference function created!")
print("\nExample usage:")
print("result = predict_from_fits('path/to/file.fits', model, scaler, feature_names)")
print("print(result['prediction'])")
print("print(result['probabilities'])")

In [ ]:
# Test the inference function on a few validation samples
print("Testing FITS-only inference on validation samples...\n")

# Load saved artifacts
scaler_loaded = joblib.load(SCALER_OUT)
feature_names_loaded = joblib.load(FEATURE_NAMES_OUT)

# Test on 5 random validation samples
test_indices = np.random.choice(val_idx, size=min(5, len(val_idx)), replace=False)

for idx in test_indices:
    fits_path = fits_paths[idx]
    true_label = int(y_arr[idx])
    true_class = ["FALSE POSITIVE", "CANDIDATE", "CONFIRMED"][true_label]
    
    result = predict_from_fits(fits_path, model, scaler_loaded, feature_names_loaded)
    
    print(f"File: {os.path.basename(fits_path)}")
    print(f"True: {true_class}")
    print(f"Predicted: {result['prediction']}")
    print(f"Probabilities: {result['probabilities']}")
    print("-" * 60)

print("\n✅ FITS-only inference is working!")

In [ ]:
# Summary
print("="*70)
print("FITS-ONLY MODEL SUMMARY")
print("="*70)
print(f"Model saved to: {MODEL_OUT}")
print(f"Scaler saved to: {SCALER_OUT}")
print(f"Feature names saved to: {FEATURE_NAMES_OUT}")
print(f"\nNumber of features: {len(feature_names_loaded)}")
print(f"Features: {feature_names_loaded}")
print(f"\nBest validation accuracy: {best_val_acc:.4f}")
print(f"\nKey Advantage: This model requires ONLY a FITS file for prediction!")
print("No KOI catalog features needed.")
print("="*70)